Starting with reading the datasets created for performance analysis in inception V3.ipynb notebook 

In [1]:
import h5py
hdf5File = h5py.File(r'..\..\data\processed\predictions.hdf5', 'r')
list(hdf5File.keys())

['top5Pred3D', 'topPred2D']

Nice! let's read the topPred2D dataset that contains the Top-1, 10-Crop image prediction

In [4]:
Prediction2D = hdf5File['topPred2D']
Prediction2D[:5]

array([[ 0.,  0., 85., 85., 20., 85., 85.,  0., 20.,  0., 85., 85.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., 52., 50., 50., 52., 52., 52., 52., 50.,  0., 52.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [12]:
%%time
from tqdm import tqdm
right_counter = 0
for i in tqdm(range(len(Prediction2D))):
    if Prediction2D[i][0] == Prediction2D[i][11]:
        right_counter += 1
        
print('Top-1 Accuracy, 10-Crop: {0:.2f}%'.format(right_counter / len(Prediction2D) * 100))

100%|██████████████████████████████████████████████████████████████████████████| 25250/25250 [00:04<00:00, 5753.34it/s]


Top-1 Accuracy, 10-Crop: 78.72%
Wall time: 4.42 s


We are getting 78.72% accuracy in first trial. Now let's see the Top-5 accuracy, 10 crops. We are assuming that both topPred2D top5Pred3d contains prediction in same order for source image.

In [16]:
Prediction3DTop5 = hdf5File['top5Pred3D']
Prediction3DTop5[:2]

array([[[52., 86., 85.,  2.,  0.],
        [ 0., 97., 58.,  2., 85.],
        [20., 52., 76., 85.,  0.],
        [ 0., 52., 74., 20., 85.],
        [52.,  9.,  2., 85.,  0.],
        [97., 52., 85., 92.,  0.],
        [86., 85.,  2., 52.,  0.],
        [85., 74.,  0., 52., 20.],
        [19., 76.,  2., 85.,  0.],
        [92.,  9.,  2.,  0., 85.]],

       [[52., 67., 59., 76.,  0.],
        [47., 52., 59., 76.,  0.],
        [67., 76.,  8., 62.,  0.],
        [74., 85., 76.,  8.,  0.],
        [47., 76., 62., 59.,  0.],
        [59., 47., 52., 76.,  0.],
        [52., 59., 67., 76.,  0.],
        [74.,  2.,  8., 85.,  0.],
        [76., 85., 62.,  8.,  0.],
        [84., 67., 76., 62.,  0.]]])

In [41]:
%%time
import numpy as np
topPred2D = np.zeros(5)
top_5_counter = 0

for i in tqdm(range(len(Prediction3DTop5))):
    for j in range(5):
        counts = np.bincount(list(np.transpose(Prediction3DTop5[i])[j]))
        mostCommonPrediction = np.argmax(counts)
        topPred2D[j] =  mostCommonPrediction
        
    if (Prediction2D[i][0] in topPred2D):
        top_5_counter += 1
        
print('Top-5 Accuracy, 10-Crop: {0:.2f}%'.format(top_5_counter / len(Prediction2D) * 100))

100%|██████████████████████████████████████████████████████████████████████████| 25250/25250 [00:15<00:00, 1677.52it/s]


Top-5 Accuracy, 10-Crop: 87.28%
Wall time: 15.1 s


For Top-5 accuracy we get 87.28%. Which is a big jump. For calculating the value we used the most common value among same index of top-5 prediction for each crop. We can use another technique where we use all the values in top-5 and find the top 5 most common values of it. 

In [75]:
%%time
from collections import Counter
topPred2D = np.zeros(5)
top_5_counter = 0

for i in tqdm(range(len(Prediction3DTop5))):
    for j in range(5):
        mostCommonFlattenedList = Counter(list(Prediction3DTop5[i].flatten())).most_common(5)
        topPred2D[j] =  mostCommonFlattenedList[j][0]
      
    if (Prediction2D[i][0] in topPred2D):
        top_5_counter += 1
        
print('Top-5 Accuracy, 10-Crop: {0:.2f}%'.format(top_5_counter / len(Prediction2D) * 100))

100%|██████████████████████████████████████████████████████████████████████████| 25250/25250 [00:17<00:00, 1428.50it/s]


Top-5 Accuracy, 10-Crop: 93.53%
Wall time: 17.7 s


Nice!! Got a jump of more than 6% accuracy. 